In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Define the directory containing your CSV files
data_dir = '/kaggle/input/sec-cameco/' if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ else '../data/tables_1009001/'

for dirname, _, filenames in os.walk(data_dir):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

../data/tables_1009001/
../data/tables_1009001/dim.csv
../data/tables_1009001/tag.csv
../data/tables_1009001/pre.csv
../data/tables_1009001/num.csv
../data/tables_1009001/ren.csv
../data/tables_1009001/sub.csv
../data/tables_1009001/cal.csv
../data/tables_1009001/txt.csv


In [21]:
import duckdb
import os

sec_pk = {'sub': 'adsh', 'tag': 'tag, version', 'ren': 'adsh, report', 'pre': 'adsh, report, line', 'cal': 'adsh, grp, arc', 'dim': 'dimhash'}
# Create an in-memory DuckDB connection
con = duckdb.connect(database=':memory:', read_only=False)

# List all files in the specified directory
all_files = os.listdir(data_dir)

# Filter for CSV files (you might need to adjust the extension if your files are different)
csv_files = [f for f in all_files if f.endswith('.csv')]

# Iterate through the CSV files and load them into DuckDB tables
for file_name in csv_files:
    file_path = os.path.join(data_dir, file_name)
    table_name = os.path.splitext(file_name)[0]  # Use the filename (without extension) as the table name

    try:
        # Option 1: Use pandas to read the CSV file into a DataFrame
        #df = pd.read_csv(file_path)

        # Load the DataFrame into a DuckDB table
        #con.register(table_name, df)  # Register the DataFrame as a view

        # Option 2: Using COPY FROM (Faster for large files)
        con.execute(f"""CREATE TABLE IF NOT EXISTS {table_name} AS 
                    SELECT
                        strptime(raw_date_int, '%Y%m%d')::DATE AS event_date,
                        -- Other columns
                        * EXCLUDE (raw_date_int)
                    FROM read_csv('{file_path}', AUTO_DETECT=TRUE);""")
        if table_name in sec_pk:
            # Add primary key constraint
            con.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({sec_pk[table_name]});")

        print(f"Loaded '{file_name}' into DuckDB view '{table_name}'")

    except Exception as e:
        print(f"Error loading '{file_name}': {e}")

# Now you can query the data in DuckDB using SQL
# For example, to select the first few rows of one of your tables:
# table_to_query = csv_files[0].split('.')[0] # Get the name of the first table
# result = con.execute(f"SELECT * FROM {table_to_query} LIMIT 5;").fetchdf()
# print(result)

Error loading 'dim.csv': Binder Error: Column "raw_date_int" in EXCLUDE list not found in FROM clause
Error loading 'tag.csv': Binder Error: Column "raw_date_int" in EXCLUDE list not found in FROM clause
Error loading 'pre.csv': Binder Error: Column "raw_date_int" in EXCLUDE list not found in FROM clause
Error loading 'num.csv': Binder Error: Column "raw_date_int" in EXCLUDE list not found in FROM clause
Error loading 'ren.csv': Binder Error: Column "raw_date_int" in EXCLUDE list not found in FROM clause
Error loading 'sub.csv': Binder Error: Column "raw_date_int" in EXCLUDE list not found in FROM clause
Error loading 'cal.csv': Binder Error: Column "raw_date_int" in EXCLUDE list not found in FROM clause
Error loading 'txt.csv': Binder Error: Column "raw_date_int" in EXCLUDE list not found in FROM clause


In [5]:

def print_duckdb_schema(conn: duckdb.DuckDBPyConnection, table_name: str):
  """Prints the schema of a specified table in a DuckDB database.

  Args:
    conn: An active DuckDB connection object.
    table_name: The name of the table whose schema to print.
  """
  try:
    schema_info = conn.execute(f"PRAGMA table_info('{table_name}')").fetchall()
    if not schema_info:
      print(f"Table '{table_name}' not found.")
      return

    print(f"Schema for table: {table_name}")
    print("-" * (16 + len(table_name)))
    print(f"{'Column ID':<10} {'Name':<20} {'Type':<15} {'NotNull':<8} {'PrimaryKey':<12} {'Default':<20}")
    print("-" * 85)
    for column in schema_info:
      cid, name, dtype, notnull, pk, default = column
      print(f"{cid:<10} {name:<20} {dtype:<15} {bool(notnull):<8} {bool(pk):<12} {str(default):<20}")
    print("-" * 85)

  except duckdb.CatalogException as e:
    print(f"Error: {e}")
    print(f"Could not retrieve schema for table '{table_name}'.")

if __name__ == "__main__":
  # Example usage:

  # 1. Connect to an in-memory DuckDB database (or a file)
  conn = duckdb.connect(':memory:')
  # conn = duckdb.connect('my_database.db')

  # 2. Create a sample table
  conn.execute("""
  CREATE TABLE employees (
    id INTEGER PRIMARY KEY,
    name VARCHAR,
    age INTEGER NOT NULL,
    salary DECIMAL(10, 2),
    hire_date DATE DEFAULT CURRENT_DATE
  );
  """)

  # 3. Print the schema of the 'employees' table
  print_duckdb_schema(conn, 'employees')

  # 4. Try printing the schema of a non-existent table
  print("\nTrying to print schema of a non-existent table:")
  print_duckdb_schema(conn, 'departments')

  # 5. Close the connection (important for file-based databases)
  conn.close()

Schema for table: employees
-------------------------
Column ID  Name                 Type            NotNull  PrimaryKey   Default             
-------------------------------------------------------------------------------------
0          id                   INTEGER         1        0            True                
1          name                 VARCHAR         0        0            False               
2          age                  INTEGER         1        0            False               
3          salary               DECIMAL(10,2)   0        0            False               
4          hire_date            DATE            0        1            False               
-------------------------------------------------------------------------------------

Trying to print schema of a non-existent table:
Error: Catalog Error: Table with name departments does not exist!
Did you mean "pg_attrdef"?
Could not retrieve schema for table 'departments'.


In [22]:
result = con.execute(f"SELECT * FROM sub;")
print(result.fetchdf())

print_duckdb_schema(con, 'sub')

CatalogException: Catalog Error: Table with name sub does not exist!
Did you mean "duckdb_logs"?

LINE 1: SELECT * FROM sub;
                      ^

In [ ]:
# Remember to close the DuckDB connection when you're finished
con.close()